In [33]:
import pandas as pd
import pandas_ta as ta
import numpy as np
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from backtesting import Backtest, Strategy

In [28]:
df = pd.read_csv('data/spgi.csv')
df['Date'] = pd.to_datetime(df['Date'], utc=True)
df.set_index('Date', inplace=True)
df=df[df.High!=df.Low]

In [55]:
df.ta.macd(fast=12, slow=26, signal=9, append=True)
df['Signal'] = np.where(df['MACDh_12_26_9'] > 0, 'B', np.where(df['MACDh_12_26_9'] < 0, 'S', 'N'))
df['ATR'] = ta.atr(df.High, df.Low, df.Close, length=14)
df = df.dropna(subset=['MACDh_12_26_9', 'ATR'])
df

Open        High         Low       Close  \
Date                                                                        
2023-11-02 04:00:00+00:00  363.452398  378.209569  363.452398  374.326111   
2023-11-03 04:00:00+00:00  379.792847  383.895374  378.438598  381.276520   
2023-11-06 05:00:00+00:00  380.589447  383.676309  379.713173  383.019104   
2023-11-07 05:00:00+00:00  384.114412  386.354872  382.043245  382.939423   
2023-11-08 05:00:00+00:00  381.854062  385.100219  380.499813  384.741760   
...                               ...         ...         ...         ...   
2024-03-11 04:00:00+00:00  427.000000  428.690002  423.290009  426.679993   
2024-03-12 04:00:00+00:00  429.320007  430.730011  426.149994  428.609985   
2024-03-13 04:00:00+00:00  428.290009  430.170013  426.679993  428.029999   
2024-03-14 04:00:00+00:00  426.720001  428.100006  421.059998  423.470001   
2024-03-15 04:00:00+00:00  420.029999  424.519989  418.649994  422.809998   

                            Volume  Dividends  Stock Splits  MACD_12_26_9  \
Date                                                                        
2023-11-02 04:00:00+00:00  3046500        0.0           0.0     -4.610237   
2023-11-03 04:00:00+00:00  1888400        0.0           0.0     -2.101630   
2023-11-06 05:00:00+00:00  1209300        0.0           0.0      0.026762   
2023-11-07 05:00:00+00:00   908800        0.0           0.0      1.687645   
2023-11-08 05:00:00+00:00   781700        0.0           0.0      3.113450   
...                            ...        ...           ...           ...   
2024-03-11 04:00:00+00:00   987300        0.0           0.0     -2.672731   
2024-03-12 04:00:00+00:00  1005600        0.0           0.0     -2.412361   
2024-03-13 04:00:00+00:00   750100        0.0           0.0     -2.227143   
2024-03-14 04:00:00+00:00  1296000        0.0           0.0     -2.420409   
2024-03-15 04:00:00+00:00  2467000        0.0           0.0     -2.596895   

                           MACDh_12_26_9  MACDs_12_26_9       ATR Signal  
Date                                                                      
2023-11-02 04:00:00+00:00       2.275915      -6.886152  8.922753      B  
2023-11-03 04:00:00+00:00       3.827617      -5.929248  8.972974      B  
2023-11-06 05:00:00+00:00       4.764808      -4.738046  8.586224      B  
2023-11-07 05:00:00+00:00       5.140552      -3.452908  8.258126      B  
2023-11-08 05:00:00+00:00       5.253086      -2.139636  7.978864      B  
...                                  ...            ...       ...    ...  
2024-03-11 04:00:00+00:00       0.033896      -2.706627  6.648849      B  
2024-03-12 04:00:00+00:00       0.235413      -2.647774  6.501056      B  
2024-03-13 04:00:00+00:00       0.336505      -2.563647  6.285957      B  
2024-03-14 04:00:00+00:00       0.114591      -2.535000  6.339824      B  
2024-03-15 04:00:00+00:00      -0.049516      -2.547379  6.306261      S  

[92 rows x 12 columns]

In [58]:
fig = make_subplots(rows=2, cols=1)
fig.add_trace(go.Candlestick(x=df.index,
                             open=df['Open'],
                             high=df['High'],
                             low=df['Low'],
                             close=df['Close']), row=1, col=1)
fig.add_trace(go.Scatter(x=df.index, y=df['MACD_12_26_9'], mode='lines', name='MACD Line'), row=2, col=1)
fig.add_trace(go.Scatter(x=df.index, y=df['MACDs_12_26_9'], mode='lines', name='Signal Line'), row=2, col=1)
fig.add_trace(go.Bar(x=df.index, y=df['MACDh_12_26_9'], name='Histogram'), row=2, col=1)

fig.show()


In [57]:
class MyStrategy(Strategy):
  size = 100
  slCoefficient = 1.1
  tpCoefficient = 1.5

  def init(self):
    super().init()
    self.signal = self.data.Signal

  def next(self):
    super().next()
    # stopLossAtr = self.slCoefficient * (self.data.ATR[-1] if not np.isnan(self.data.ATR[-1]) else 6)
    stopLossAtr = self.slCoefficient * self.data.ATR[-1]

    if self.signal == 'B' and len(self.trades) == 0:
      stopLoss = self.data.Close[-1] - stopLossAtr
      takeProfit = self.data.Close[-1] + self.tpCoefficient * stopLossAtr
      self.buy(sl=stopLoss, tp=takeProfit, size=self.size)

    elif self.signal == 'S' and len(self.trades) == 0:
      stopLoss = self.data.Close[-1] + stopLossAtr
      takeProfit = self.data.Close[-1] - self.tpCoefficient * stopLossAtr
      self.sell(sl=stopLoss, tp=takeProfit, size=self.size)


bt = Backtest(df, MyStrategy, cash=1000000000, commission=.002)

results = bt.run()

# Print the backtesting results
print(results)


Start                     2023-11-02 04:00...
End                       2024-03-15 04:00...
Duration                    134 days 00:00:00
Exposure Time [%]                   97.826087
Equity Final [$]             999993815.990305
Equity Peak [$]                  1000000000.0
Return [%]                          -0.000618
Buy & Hold Return [%]               12.952312
Return (Ann.) [%]                   -0.001694
Volatility (Ann.) [%]                0.000551
Sharpe Ratio                              0.0
Sortino Ratio                             0.0
Calmar Ratio                              0.0
Max. Drawdown [%]                   -0.000802
Avg. Drawdown [%]                   -0.000802
Max. Drawdown Duration      133 days 00:00:00
Avg. Drawdown Duration      133 days 00:00:00
# Trades                                   16
Win Rate [%]                            18.75
Best Trade [%]                       3.375314
Worst Trade [%]                     -2.979905
Avg. Trade [%]                    